In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

/Users/karenzhang/Documents/Northeastern/Fall24/NLP/CS6120/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset_path = "/Users/karenzhang/Documents/Northeastern/Fall24/NLP/FinalProject/text.csv"
df = pd.read_csv(dataset_path)
df = df.drop("Unnamed: 0", axis=1)
df

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like telling these horny devils to find...,2
416805,i began to realize that when i was feeling agi...,3
416806,i feel very curious be why previous early dawn...,5
416807,i feel that becuase of the tyranical nature of...,3


In [19]:
train_df, test_df = train_test_split(df.head(100), test_size=0.2, random_state=10, shuffle=True)
train_df

,text,label
90,i don t want to sit inside and be alone i don ...,4
70,i feel that lauterbach was victimized again an...,0
50,i have wished her the best and i truly feel as...,1
44,i feel my life is totally in ecstatic chaos,1
76,i feel that the technology and the act of comm...,4
...,...,...
89,i was feeling really overwhelmed,5
28,when,3
64,im feeling a little better and with more chris...,1
15,i may be more biased than the next because i h...,1


In [20]:
from transformers import GPT2Tokenizer
from transformers import GPT2ForSequenceClassification

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 80
})

In [22]:
train_tokenized_datasets = train_dataset.map(tokenize_function, batched = True)
test_tokenized_datasets = test_dataset.map(tokenize_function, batched = True)
train_tokenized_datasets

Map: 100%|██████████| 20/20 [00:00<00:00, 3384.69 examples/s]


Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 80
})

In [23]:
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['text',  '__index_level_0__'])
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['text',  '__index_level_0__'])
train_tokenized_datasets

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 80
})

In [24]:
train_tokenized_datasets.set_format(type='torch', columns = ['input_ids', 'attention_mask', 'label'])
test_tokenized_datasets.set_format(type='torch', columns = ['input_ids', 'attention_mask', 'label'])
train_tokenized_datasets

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 80
})

In [25]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
)


In [27]:
trainer.train()

  0%|          | 0/10 [47:39:21<?, ?it/s]


RuntimeError: MPS backend out of memory (MPS allocated: 19.34 GB, other allocations: 971.78 MB, max allowed: 20.40 GB). Tried to allocate 192.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).